In [16]:
import torch
import torchvision
import pandas as pd
import os
from tqdm import tqdm
import pydicom
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from utils import normalize_dicom

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = torchvision.models.resnet18()
model.conv1 = torch.nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.fc = torch.nn.Linear(model.fc.in_features, 1)  
model = model.to(device)

model.load_state_dict(torch.load('frame_selection.pth'))

for param in model.parameters():
    param.requires_grad = False

In [18]:
test_img_ids = set(pd.read_csv("testimgs.csv").ID)
test_study_ids = set(pd.read_csv("teststudies.csv").ID)

In [55]:
df1 = pd.read_csv("data/v2/imagerejectionfeedback.csv", sep=";")
df1 = df1.set_index("IMAGE_ID")
#df = df.set_index("IMAGE_ID")
df2 = pd.read_csv("data/v2/image.csv", sep=";")
df2 = df2.set_index("ID")

df = df1.join(df2, how="inner")

df = df[df.index.isin(test_img_ids)]
df["accepted"] = np.NaN
df

,ID,TECHNICALVESSELSNOTVISIBLE,TECHNICALVESSELSOVERLAP,TECHNICALNOCONTRAST,TECHNICALLOWQUALITY,PROCEDURALFEMORALACCESS,PROCEDURALELECTRODESVISIBLE,PROCEDURALCATHETERINAORTA,EXAMINATIONAORTOGRAPHY,EXAMINATIONVENTRICULOGRAPHY,...,REPRESENTATIVEFRAME,SERIESDESCRIPTION,SERIESINSTANCEUID,SERIESNO,SEX,STUDYDESCRIPTION,STUDYINSTANCEUID,URL,STUDY_ID,accepted
IMAGE_ID,,,,,,,,,,,,,,,,,,,,,
12aw4ack71831bocuf5j3pz238tcmv77343,28a83b25-7271-4bbc-b209-994bc08369c0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,38,NaN,1.2.392.200036.9116.3.1.19612167.2022093012460...,2,M,NaN,1.2.392.200036.9116.3.1.19612167.2022093012183...,Ukraine/modified/p2s2.dcm,1.2.392.200036.9116.3.1.19612167.2022093012183...,NaN
12aw4ack71831bocuf5j3pz23ddfsvb226y,580e4158-b608-4ede-9eec-503a09027020,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,23,NaN,1.2.392.200036.9116.3.1.19612167.2022093012481...,3,M,NaN,1.2.392.200036.9116.3.1.19612167.2022093012183...,Ukraine/modified/p2s3.dcm,1.2.392.200036.9116.3.1.19612167.2022093012183...,NaN
12aw4ack71831bocuf5j3pz23f7d6vf0362,9a5e7988-4d46-4739-85ed-687585d4da4b,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,45,NaN,1.2.392.200036.9116.3.1.19612167.2022093012490...,4,M,NaN,1.2.392.200036.9116.3.1.19612167.2022093012183...,Ukraine/modified/p2s4.dcm,1.2.392.200036.9116.3.1.19612167.2022093012183...,NaN
12aw4ack71831bocuf5j3pz23jx4qvj87o6,bae23a2e-a1c9-4227-976e-eccbd67de623,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,36,NaN,1.2.392.200036.9116.3.1.19612167.2022093012512...,5,M,NaN,1.2.392.200036.9116.3.1.19612167.2022093012183...,Ukraine/modified/p2s5.dcm,1.2.392.200036.9116.3.1.19612167.2022093012183...,NaN
12aw4ack71831bocuf5j3pz23k1ttvn84cw,ed793fac-78e1-4803-b0b5-db54120f934f,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,36,NaN,1.2.392.200036.9116.3.1.19612167.2022093012512...,6,M,NaN,1.2.392.200036.9116.3.1.19612167.2022093012183...,Ukraine/modified/p2s6.dcm,1.2.392.200036.9116.3.1.19612167.2022093012183...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13c2ur549vohc0jat2ewj4eycc2,dcd19103-5106-4f34-96a7-4a573b39fb81,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,38,NaN,1.3.12.2.1107.5.4.9.41057.20180405.112627.7416...,1,M,NaN,1.3.12.2.1107.5.4.9.41057.20180405.112627.7416...,data/05/Coronary_Diagnostic_Coronary_Catheteri...,1.3.12.2.1107.5.4.9.41057.20180405.112627.7416...,NaN
13c2ur549vohc0jat2ewj4eyk51,f4673173-4331-4c25-9c5c-889ced6a24d0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,33,NaN,1.3.12.2.1107.5.4.9.41057.20180405.112627.7416...,3,M,NaN,1.3.12.2.1107.5.4.9.41057.20180405.112627.7416...,data/05/Coronary_Diagnostic_Coronary_Catheteri...,1.3.12.2.1107.5.4.9.41057.20180405.112627.7416...,NaN
13c2ur549vohc0jat2ewk4eyu41,da52624d-f7c2-4977-ac37-acc1c1b66577,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,35,NaN,1.3.12.2.1107.5.4.9.41057.20180405.112628.7416...,6,M,NaN,1.3.12.2.1107.5.4.9.41057.20180405.112627.7416...,data/05/Coronary_Diagnostic_Coronary_Catheteri...,1.3.12.2.1107.5.4.9.41057.20180405.112627.7416...,NaN


In [67]:
images = {}

parent = "data/series_selection/rejected/dicoms/"
dcm_files = os.listdir(parent)
dcm_files = [f for f in dcm_files if f.endswith(".dcm")]
dcm_files = [f for f in dcm_files if f.replace(".dcm", "") in test_img_ids]
for filename in tqdm(dcm_files):
    # dcm = pydicom.dcmread(parent + filename)
    # img = normalize_dicom(dcm)
    # images[filename.replace(".dcm", "")] = img
    df.loc[filename.replace(".dcm", ""), "accepted"] = 0

parent = "data/series_selection/accepted/dicoms/"
dcm_files = os.listdir(parent)
dcm_files = [f for f in dcm_files if f.endswith(".dcm")]
dcm_files = [f for f in dcm_files if f.replace(".dcm", "") in test_img_ids]
for filename in tqdm(dcm_files):
    # dcm = pydicom.dcmread(parent + filename)
    # img = normalize_dicom(dcm)
    # images[filename.replace(".dcm", "")] = img
    df.loc[filename.replace(".dcm", ""), "accepted"] = 1


  0%|                                                                                                                                    | 0/106 [00:00<?, ?it/s]/tmp/ipykernel_462050/1920176458.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[filename.replace(".dcm", ""), "accepted"] = 1
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 106/106 [00:00<00:00, 707.05it/s]


In [104]:
df = df.dropna(subset=["accepted", "ID"])

# df = df.join(
#     df.groupby("STUDY_ID").mean("accepted")["accepted"] == 0,
#     on="STUDY_ID", rsuffix="_to_drop", how="inner"
# )

# df = df[~df["accepted_to_drop"]]
# df = df.drop(columns=["accepted_to_drop"])
# df

In [16]:
def select_series(study_id, method="max"):
    images_df = df[df["STUDY_ID"] == study_id]

    prob_list = []
    for img_id in images_df.index:
        img = images[img_id]
        img = torch.tensor(img).to(device).unsqueeze(1)
        probs = torch.sigmoid(model(img))
        probs = probs.squeeze(1).cpu().numpy()
        prob_list.append(probs)

    if method == "max":
        prob_list = [a.max() for a in prob_list]
    elif method == "mean":
        prob_list = [a.mean() for a in prob_list]
    elif method == "top3":
        prob_list = [a[np.argsort(-a)[:3]].mean() for a in prob_list]

    prob_list = np.array(prob_list)
    order = np.argsort(-prob_list)
    return images_df.index.values[order], prob_list[order]

In [129]:
studies = df["STUDY_ID"].unique()
random_study_id = np.random.choice(studies)
print(random_study_id)
df[(df["STUDY_ID"]==random_study_id)]["accepted"]

1.3.46.670589.28.68172260235162120211203195849675694


IMAGE_ID
131aedfhs6pnf1fvtvp49jzll6tptz4q22    0.0
131aedfhs6pnf1fvtvp49jzll73sqdtm22    0.0
131aedfhs6pnf1fvtvp49jzll76yd5ku22    0.0
131aedfhs6pnf1fvtvp49jzll78ru67c22    0.0
Name: accepted, dtype: float64

In [17]:
studies = df["STUDY_ID"].unique()
random_study_id = np.random.choice(studies)
accepted = df.loc["STUDY_ID"[df["accepted"]==True]
select_series(random_study_id)

(array(['131aedfhs6pnf1fvtvp49jzll78ru67c22',
        '131aedfhs6pnf1fvtvp49jzll6tptz4q22',
        '131aedfhs6pnf1fvtvp49jzll76yd5ku22',
        '131aedfhs6pnf1fvtvp49jzll73sqdtm22'], dtype=object),
 array([0.7460165 , 0.7360174 , 0.73365164, 0.6341753 ], dtype=float32))

In [ ]:
def top_k_accuracy():
    pos = []
    for study_id in studies:
        accepted = df["
        ranking, scores = select_series(study_id)
        pos = 